# Demo RNA Multi-Perceptrón Backpropagation para identificar TIPOS de ANIMALES
Adaptado de https://www.tensorflow.org/tutorials/estimator/premade

1) Cargar librerías:

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
from numpy.random import RandomState

from  sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("Librerías cargadas")

2) Cargar los datos:

In [ ]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demo ANIMALES'

In [ ]:
# Carga los datos del CSV y muestra los primeros
df = pd.read_csv(path + '/datos/animales.csv')

df.head()

In [ ]:
# define clases (ejemplo ANIMALES)
SPECIES = ['na', 'MAMIFERO', 'AVE', 'REPTIL', 'PEZ', 'ANFIBIO', 'INSECTO', 'INVERTEBRADO']

# define nombre atributo de CLASE (ejemplo ANIMALES)
ClassAttributeName = 'TIPO'

print("Configuración definida de ", ClassAttributeName, ": ", SPECIES)

In [ ]:
# separa al azar con proporción 80/20
rng = RandomState()

train = df.sample(frac=0.8, random_state=rng)
test = df.loc[~df.index.isin(train.index)]

print("Datos Originales ", df.shape)
print("- Datos para Entrenar ", train.shape)
print("- Datos para Probar ", test.shape)

# genera los datos solo con la clase para entrenar y probar
train_y = train.pop(ClassAttributeName)
test_y = test.pop(ClassAttributeName)

3) Definir funciones auxiliares para el entrenamiento:

In [ ]:
# funciones auxiliares para entrenar y probar 
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

    
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

4) Establecer el modelo para la RNA

In [ ]:
# Construye un estimador tipo RNA 'Deep' Neuronal Network classifier
classifier = tf.estimator.DNNClassifier(
    
    feature_columns=my_feature_columns,

    hidden_units=[35, 25, 15], # 3 capas de 30, 20 y 10 neuronas ocultas    
    
    n_classes=len(SPECIES)) # identifica clases



5) Entrenar el modelo de la RNA:

In [ ]:
# Entrena el modelo
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=20000) 

6) Evaluar el modelo de la RNA entrenado:

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nExactitud: {accuracy:0.3f}\n'.format(**eval_result))

7) Mostrar el detalle con predicciones sobre datos de Prueba:

In [ ]:
# funciones auxiliares
def val_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: val_fn(test))


# muestra resultados de la predicción
classPreds = []
classReal = []
cantOK = 0
cantError = 0
print("\n Resultados: ")
for pred_dict, expec in zip(predictions, test_y):
    pred_class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][pred_class_id]
    
    classPreds.append(SPECIES[pred_class_id])
    classReal.append(SPECIES[expec])

    if (pred_class_id == expec): 
      res = ""
      cantOK += 1
    else: 
      res = "!"
      cantError += 1

    print('Clase predecida es "{}"[{}] ({:.1f}%), la correcta es "{}"[{}]{} '.format(
        SPECIES[pred_class_id], pred_class_id, 100 * probability, SPECIES[expec], expec, res))

print('=== Total Ejemplos: {}, con {} predicciones ok y {} errores.'.format(cantOK+cantError, cantOK, cantError))

# gráfico de comparación
plt.title('Gráfico de Confusión')
plt.xlabel('Real')
plt.ylabel('RNA')
plt.scatter(classReal, classPreds)

# muestra reporte de clasificación
print("\n Reporte de Clasificación: ")
print(classification_report(classReal, classPreds))

# muestra matriz de confusion
print('\nMatriz de Confusión: ')
cm = confusion_matrix(classReal, classPreds, labels=SPECIES)
cmtx = pd.DataFrame(
    cm, 
    index=['r:{:}'.format(x) for x in SPECIES], 
    columns=['p:{:}'.format(x) for x in SPECIES]
  )
print(cmtx)
print("\n")
